### 几个网络模块

#### socket套接字

服务器套接字和客户端套接字

创建服务器套接字后，让它等待连接请求的到来。这样，它将在某个网络地址（由IP地址和端口号组成）处监听，直到客户端套接字建立连接。随

服务器必须准备随时处理客户端连接，还必须处理多个连接；而客户端只需连接，完成任务后再断开连接即可

套接字是模块socket中socket类的实例。实例化套接字时最多可指定三个参数：

- 一个地址族（默认为socket.AF_INET）

- 流套接字（socket.SOCK_STREAM，默认设置）

  数据报套接字（socket.SOCK_DGRAM）；

- 协议（使用默认值0就好）。创建普通套接字时，不用提供任何参数



socket创建套接字服务器和客户端

### 服务器模块

SocketServer是标准库提供的服务器框架的基石，这个框架包括BaseHTTPServer、SimpleHTTPServer、CGIHTTPServer、SimpleXMLRPCServer和DocXMLRPCServer等服务器，它们在基本服务器的基础上添加了各种功能。

使用模块SocketServer编写服务器时，大部分代码都位于请求处理器中。每当服务器收到客
户端的连接请求时，都将实例化一个请求处理程序，并对其调用各种处理方法来处理请求

基本请求处理程序类BaseRequestHandler将所有操作都放在一个方法中——服务器调用的方法handle。

这个方法可通过属性self.request来访问客户端套接字。

如果处理的是流（使用TCPServer时很可能如此），可使StreamRequestHandler类，

它包含另外两个属性：self.rfile（用于读取）和self.wfile（用于写入）。

你可使用这两个类似于文件的对象来与客户端通信。

In [ ]:
#服务端
import socket
s = socket.socket()
host = socket.gethostname()
port = 1234
s.bind((host, port))#1.绑定
s.listen(5)#2.监听
while True:
    c, addr = s.accept()#3.接收
    print('Got connection from', addr)
    c.send(byte('Thank you for connecting',"utf-8"))
    c.close()
    

In [ ]:
#客户端
import socket
s = socket.socket()
host = socket.gethostname()
port = 1234
s.connect((host, port))#
print(s.recv(1024))

In [ ]:
#14.3创建多线程类型的TCP SocketServer
import socketserver

class MyTCPHandler(socketserver.BaseRequestHandler):
    """
　　 客户端发送a-z字符串，socketserver返回大写
    """

    def handle(self):    #必须要有handle方法；所有处理必须通过handle方法实现
        # self.request is the TCP socket connected to the client
        while True:
            self.data = self.request.recv(1024).strip()
            if not self.data:
                print('The client disconnects actively!')
                break
            self.return_client()

    def return_client(self):    #数据处理方法
        print("Ip:{0} Port{1}:".format(self.client_address[0], self.client_address[1]))
        print(self.data.decode('utf8'))
        # just send back the same data, but upper-cased
        self.request.sendall(self.data.upper())


if __name__ == "__main__":
    HOST, PORT = "127.0.0.1", 9990

    # Create the server, binding to localhost on port 9999
    server = socketserver.ThreadingTCPServer((HOST, PORT), MyTCPHandler)    #实例化一个多线程TCPServer
    print('Wait client . . .') 
    server.serve_forever()

In [ ]:
#14.4TCPsocketClient.py
import socket


HOST, PORT = "127.0.0.1", 9990

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

while True:
    data = input('Input [a-z]:>>')
    if data == 'bye': break
    sock.sendall(bytes(data + "\n", "utf-8"))
    received = str(sock.recv(1024), "utf-8")
    print("Sent:     {}".format(data))
    print("Received: {}".format(received))
sock.close()